In [1]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2022-12-13 14:02:06--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M   494KB/s    in 6.0s    

2022-12-13 14:02:13 (826 KB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [3]:
df = pd.read_csv("bbc_text_cls.csv")

In [4]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [5]:
df["targets"] = df["labels"].astype("category").cat.codes

In [6]:
K = df["targets"].max() + 1
K

5

In [7]:
df_train, df_test = train_test_split(df, test_size=0.33)

In [8]:
MAX_VOCAB_SIZE = 2000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train["text"])
sequences_train = tokenizer.texts_to_sequences(df_train["text"])
sequences_test = tokenizer.texts_to_sequences(df_test["text"])

In [9]:
word2idx = tokenizer.word_index
V = len(word2idx)
print(f"Found {V} unique tokens.")

Found 27397 unique tokens.


In [11]:
data_train = pad_sequences(sequences_train)
print(f"Shape of data train tensor {data_train.shape}")

T = data_train.shape[1]

Shape of data train tensor (1490, 3479)


In [12]:
data_test = pad_sequences(sequences_test, maxlen=T)
print(f"Sahpe of data test tensor {data_test.shape}")

Sahpe of data test tensor (735, 3479)


In [13]:
D = 50
i = Input(shape=(T,))
x = Embedding(V+1, D)(i)
x = Conv1D(32, 3, activation="relu")(x)
# x = MaxPooling1D(3)(x)
# x = Conv1D(64, 3, activation="relu")(x)
# x = MaxPooling1D(3)(x)
# x = Conv1D(128, 3, activation="relu")(x)
x = GlobalMaxPooling1D()(x)
x = Dense(K)(x)

model = Model(i, x)

In [14]:
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"]
)

ft = model.fit(
    data_train, df_train["targets"],
    validation_data=(data_test, df_test["targets"]),
    epochs=50
)

Epoch 1/50
47/47 [==============================] - 11s 217ms/step - loss: 1.5650 - accuracy: 0.4289 - val_loss: 1.4934 - val_accuracy: 0.6435
Epoch 2/50
47/47 [==============================] - 10s 213ms/step - loss: 1.3284 - accuracy: 0.7624 - val_loss: 1.1735 - val_accuracy: 0.7823
Epoch 3/50
47/47 [==============================] - 16s 347ms/step - loss: 0.8781 - accuracy: 0.8799 - val_loss: 0.6981 - val_accuracy: 0.8830
Epoch 4/50
47/47 [==============================] - 19s 406ms/step - loss: 0.4572 - accuracy: 0.9208 - val_loss: 0.4168 - val_accuracy: 0.9102
Epoch 5/50
47/47 [==============================] - 18s 377ms/step - loss: 0.2663 - accuracy: 0.9436 - val_loss: 0.3022 - val_accuracy: 0.9224
Epoch 6/50
47/47 [==============================] - 15s 329ms/step - loss: 0.1795 - accuracy: 0.9617 - val_loss: 0.2434 - val_accuracy: 0.9293
Epoch 7/50
47/47 [==============================] - 10s 212ms/step - loss: 0.1299 - accuracy: 0.9765 - val_loss: 0.2134 - val_accuracy: 0.9347

KeyboardInterrupt: ignored

In [ ]:
plt.plot(ft.history["loss"], label="train loss")
plt.plot(ft.history["val_loss"], label="val loss")
plt.legend();

In [ ]:
plt.plot(ft.history["accuracy"], label="train acc")
plt.plot(ft.history["val_accuracy"], label="val acc")
plt.legend();